# Problem Statement

## Q1. Linear Programming

Your start-up will face the cash requirements shown in Table 1 in the next eight quarters (positive entries represent cash needs while negative entries represent cash surpluses). The company has three borrowing possibilities.

**Table 1: Cash Flow (in Crores of INR)**

| Q1  | Q2  | Q3  | Q4   | Q5   | Q6  | Q7  | Q8   |
| --- | --- | --- | ---- | ---- | --- | --- | ---- |
| 100 | 500 | 100 | -600 | -500 | 200 | 600 | -900 |

**Borrowing Possibilities:**

*   A 2-year loan available at the beginning of Q1, with a 1% interest per quarter.
*   The other two borrowing opportunities are available at the beginning of every quarter:
    *   A 6-month loan with a 1.8% interest per quarter.
    *   A quarterly loan with a 2.5% interest for the quarter.
*   Any surplus can be invested at a 0.5% interest per quarter.

**Questions:**

a) Formulate a LP that maximizes the wealth of the company at the beginning of Q9.

b) Use built-in LP solvers to get a solution.

c) What is a good initial basic feasible point?

d) Use the revised simplex method and see if your solution matches the one from built-in solver. Explain.

# My approach
## Assumptions
- <span style="color:blue">We have to pay the costs of the quarter before our investment for the quarter matured.</span>
- <span style="color:blue">We pay simple interest of the corresponding rate to the banks each month until full repayment</span>
- <span style="color:blue">We cannot take a 6 month loan at the beginning of the 8th quarter as this will be repayed after the ninth quarter</span>
- <span style="color:blue">The interest on out investments is available before we have to repay our loans</span>
- <span style="color:blue">Fresh loans can be raised only after the interest payment of existing loans.</span>

## Variables

- $x_t$: Total money at the start of quarter $t$ **after** receiving loans  
- $L$: Two-year loan available at the beginning of the first quarter  
- $j_t$: 6-month loan available at the beginning of each quarter $t$  
- $k_t$: 4-month loan available at the beginning of each quarter $t$  
- $a_t$: Total money at the start of quarter $t$ **before** receiving loans, after all costs and loan repayments from the previous quarter  
- $i_t$: Investment made in quarter $t$  
- $c_t$: Net cash flow in quarter $t$  
- $r$: Final result (e.g., total return) to be maximized


## Constraints

### Non-negativity constraints
All variables must be non-negative:

- $a_t \geq 0$, $x_t \geq 0$, $j_t \geq 0$, $k_t \geq 0$, $i_t \geq 0$

All $c_t$ values must match the given values

### Initial condition
- $a_1 = 0$

### Quarter 1 constraints
- $x_1 = L + j_1 + k_1$
- $i_1 \leq x_1$
- $c_1 \leq x_1 - i_1$
- $a_2 = x_1 - c_1 + 1.05 \cdot i_1 - i_1 - 0.01 \cdot L - 0.018 \cdot j_1 - 1.025 \cdot k_1$

*(Note: This expression assumes interest is applied before repayment. If loans must be repaid before interest is applied, this expression will change and require an additional non-negativity constraint.)*

### General constraints (for $t = 2$ to $8$)
- $x_t = a_t + j_t + k_t$
- $i_t \leq x_t$
- $c_t  \leq x_t - i_t$  
- $a_{t+1} = x_t + 1.005 \cdot i_t - i_t - c_t - 1.018 \cdot j_{t-1} - 0.01 \cdot L - 0.018 \cdot j_t - 1.025 \cdot k_t$

### Final quarter constraint
- $j_8 = 0$  
  *(No new 6-month loan in the final quarter)*

### Objective
- $r = a_9 - L$  
  *(Maximize $r$, the net return after repaying the two-year loan)*


In [2]:
import numpy as np
import pulp

# Define the problem
lp_problem = pulp.LpProblem("Cash_Flow_Optimization", pulp.LpMaximize)

# Define variables
L = pulp.LpVariable("L", lowBound=0)  # 2-year loan at Q1
j = [pulp.LpVariable(f"j{t}", lowBound=0) for t in range(1, 9)]  # 6-month loans
k = [pulp.LpVariable(f"k{t}", lowBound=0) for t in range(1, 9)]  # quarterly loans
i = [pulp.LpVariable(f"i{t}", lowBound=0) for t in range(1, 9)]  # investments
x = [pulp.LpVariable(f"x{t}", lowBound=0) for t in range(1, 9)]  # money at start after loans
a = [pulp.LpVariable(f"a{t}", lowBound=0) for t in range(1, 10)]  # money before loans
r = pulp.LpVariable("r")  # final result to maximize

# Given cash requirements (positive = needs, negative = surplus)
c = [100, 500, 100, -600, -500, 200, 600, -900]

# Initial condition
lp_problem += a[0] == 0

# Quarter 1 constraints
lp_problem += x[0] == L + j[0] + k[0]
lp_problem += i[0] <= x[0]
lp_problem += c[0] <= x[0] - i[0]
lp_problem += a[1] == x[0] - c[0] + 0.005 * i[0] - 0.01 * L - 0.018 * j[0] - 1.025 * k[0]

# Constraints for quarters 2 to 8
for t in range(1, 8):
    lp_problem += x[t] == a[t] + j[t] + k[t]
    lp_problem += i[t] <= x[t]
    lp_problem += c[t] <= x[t] - i[t]
    
    if t == 7:  # Quarter 8: special case for the final quarter
        lp_problem += a[t+1] == x[t] - c[t] + 0.005 * i[t] - 0.01 * L - 1.025 * k[t] - 1.018 * j[t-1]
    else:  # Quarters 2-7
        lp_problem += a[t+1] == x[t] - c[t] + 0.005 * i[t] - 0.01 * L - 0.018 * j[t] - 1.025 * k[t] - 1.018 * j[t-1]

# Final quarter constraint: No 6-month loan in Q8
lp_problem += j[7] == 0

# Objective function: Maximize wealth at Q9 after repaying the 2-year loan
lp_problem += r == a[8] - L
lp_problem += r  # This is the objective to maximize

# Solve the problem
lp_problem.solve()

# Print the results
print(f"Status: {pulp.LpStatus[lp_problem.status]}")
print(f"Optimal wealth at Q9: {pulp.value(lp_problem.objective)}")
print("\nOptimal strategy:")
print(f"2-year loan (L): {L.value()}")
for t in range(8):
    print(f"Quarter {t+1}:")
    print(f"  6-month loan (j{t+1}): {j[t].value()}")
    print(f"  Quarterly loan (k{t+1}): {k[t].value()}")
    print(f"  Investment (i{t+1}): {i[t].value()}")
    print(f"  Money available at start (x{t+1}): {x[t].value()}")
    print(f"  Money before loans (a{t+1}): {a[t].value() if t > 0 else 0}")
print(f"Final balance (a9): {a[8].value()}")
print(f"Final result (r): {r.value()}")

# Perform additional checks to verify constraints
print("\nConstraint verification:")
for t in range(8):
    cash_flow = (x[t].value() - i[t].value() >= c[t])
    print(f"Q{t+1} cash flow constraint satisfied: {cash_flow}")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/harshit/anaconda3/envs/ml/lib/python3.12/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/eb559f510eb541e9a173abc4d737c778-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/eb559f510eb541e9a173abc4d737c778-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 40 COLUMNS
At line 165 RHS
At line 201 BOUNDS
At line 203 ENDATA
Problem MODEL has 35 rows, 43 columns and 123 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 31 (-4) rows, 39 (-4) columns and 116 (-7) elements
Perturbing problem by 0.001% of 6.3265383 - largest nonzero change 0.000575088 ( 0.0090900895%) - largest zero change 0.00020891801
0  Obj -0.43668116 Primal inf 28170.28 (18) Dual inf 0.75801005 (1)
0  Obj -0 Primal inf 28170.28 (18) Dual inf 1.0578422e+12 (15)
26  Obj 458.03736
Optimal - objective value 458.03736
After 